In [2]:
from pathlib import Path
import utils.contour_analysis as iau
import importlib
importlib.reload(iau)

# Centralized sonar defaults (inserted by sweep)
from utils.config import SONAR_VIS_DEFAULTS
config = SONAR_VIS_DEFAULTS.copy()
# Backwards-compatible variable names used in older notebooks
RANGE_MIN_M = config['range_min_m']
RANGE_MAX_M = config['range_max_m']
DISPLAY_RANGE_MAX_M = config['display_range_max_m']

# Simple Image Analysis with CV2

This notebook demonstrates:
1. **Pick a frame** from NPZ files and save it locally
2. **Use standard cv2 functions** directly for image processing
3. **Experiment** with different OpenCV operations

In [3]:
# Prefer selecting by bag ID (part of the NPZ filename) instead of a numeric index.
# Set TARGET_BAG to a substring that appears in the NPZ filename 
# '2024-08-22_14-47-39' #1
# '2024-08-22_14-29-05' #2
# '2024-08-20_14-22-12' #3
# '2024-08-20_14-31-29' #4
# '2024-08-20_18-47-40' #5
# '2024-08-20_13-55-34' #6
# '2024-08-20_13-57-42' #7
# '2024-08-20_17-02-00' #8
TARGET_BAG = '2024-08-20_14-31-29'  # change this to your desired bag ID
from utils.config import EXPORTS_DIR_DEFAULT, EXPORTS_SUBDIRS
EXPORTS_FOLDER = Path(EXPORTS_DIR_DEFAULT)

# If you keep your raw .bag files or raw data on an external drive, point DATA_DIR there.
# Example (your external disk): /Volumes/LaCie/SOLAQUA/raw_data
DATA_DIR = Path("/Volumes/LaCie/SOLAQUA/raw_data")
print(f"Using DATA_DIR = {DATA_DIR}")

# NOTE: To export data for this specific bag, use:
# python scripts/solaqua_export.py --data-dir /Volumes/LaCie/SOLAQUA/raw_data --exports-dir /Volumes/LaCie/SOLAQUA/exports --bag-stem 2024-08-20_17-02-00 --all

# Find NPZ files and pick the one matching TARGET_BAG
files = iau.get_available_npz_files()
if not files:
    raise FileNotFoundError(f"No NPZ files found in configured exports outputs (looked under {EXPORTS_FOLDER / EXPORTS_SUBDIRS.get('outputs','outputs')})")
matches = [p for p in files if TARGET_BAG in p.name]
if not matches:
    # Help the user by listing available NPZ files
    print(f'No NPZ file matched TARGET_BAG={TARGET_BAG!r}')
    print('Available NPZ files:')
    for i,p in enumerate(files):
        print(f'  {i}: {p.name}')
    raise ValueError(f'No NPZ file contains "{TARGET_BAG}" in its name')

# If multiple matches, choose the most recently modified one
selected = max(matches, key=lambda p: p.stat().st_mtime)
NPZ_FILE_INDEX = files.index(selected)

Using DATA_DIR = /Volumes/LaCie/SOLAQUA/raw_data


## Distance Analysis Over Time

Now let's perform a comprehensive analysis of the red line distance over time. The red line represents the major axis of the detected elongated contour (likely a fishing net), and we'll track how this distance changes throughout the video sequence.

In [4]:
# Create the unified processor with corridor splitting enabled
from utils.config import TRACKING_CONFIG
TRACKING_CONFIG['use_corridor_splitting'] = True  # Force enable

processor = iau.SonarDataProcessor()

# Import video generation module
import utils.video_generation as sg

# Use the enhanced video creation with CORE processor
# The processor maintains masks internally and will render them
video_path = sg.create_enhanced_contour_detection_video_with_processor(
    npz_file_index=NPZ_FILE_INDEX,          
    frame_start=1,           
    frame_count=1500,         
    frame_step=1,            
    output_path=Path(EXPORTS_DIR_DEFAULT) / EXPORTS_SUBDIRS.get('videos','videos') / 'core_aoi_tracking.mp4',
    processor=processor      
)

=== ENHANCED VIDEO CREATION WITH AOI/CORRIDOR VISUALIZATION ===
Processing 1059 frames...
AOI/Corridor visualization: ENABLED (Green ellipse + Orange corridor)
Processed 10/1059 frames
Processed 20/1059 frames
Processed 30/1059 frames
Processed 40/1059 frames
Processed 50/1059 frames
Processed 60/1059 frames
Processed 70/1059 frames
Processed 80/1059 frames
Processed 90/1059 frames
Processed 100/1059 frames
Processed 110/1059 frames
Processed 120/1059 frames
Processed 130/1059 frames
Processed 140/1059 frames
Processed 150/1059 frames
Processed 160/1059 frames
Processed 170/1059 frames
Processed 180/1059 frames
Processed 190/1059 frames
Processed 200/1059 frames
Processed 210/1059 frames
Processed 220/1059 frames
Processed 230/1059 frames
Processed 240/1059 frames
Processed 250/1059 frames
Processed 260/1059 frames
Processed 270/1059 frames
Processed 280/1059 frames
Processed 290/1059 frames
Processed 300/1059 frames
Processed 310/1059 frames
Processed 320/1059 frames
Processed 330/105

In [5]:
engine = iau.DistanceAnalysisEngine()
net_analysis_results = engine.analyze_npz_sequence(
    npz_file_index=NPZ_FILE_INDEX,    
    frame_start=1,        # Start from frame 1
    frame_count=1500,      # Analyze all frames from the video
    frame_step=1,          # Every frame
    save_outputs=True   # Save outputs to CSV and NPZ
)

=== DISTANCE ANALYSIS FROM NPZ ===
Processing 1059 frames from 1 (step=1)

=== ANALYSIS COMPLETE ===
Total frames: 1059, Successful: 1059 (100.0%)
Distance (meters): mean=2.009, std=0.290
Analysis results saved to: /Volumes/LaCie/SOLAQUA/exports/outputs/2024-08-20_14-31-29_data_cones_analysis_results.csv


In [6]:
net_analysis_results.columns

Index(['frame_index', 'timestamp', 'distance_pixels', 'angle_degrees',
       'distance_meters', 'detection_success', 'tracking_status', 'area',
       'aspect_ratio', 'ellipse_elongation', 'rect', 'centroid_x',
       'centroid_y', 'ellipse_mask', 'corridor_mask'],
      dtype='object')

## Convert to Real-World Distances

Now let's convert the pixel distances to real-world distances using the fact that the entire sonar image represents a 10x10 meter area.

In [7]:
# Auto-detect pixel->meter mapping from the selected NPZ using utils function
mapping_info = iau.get_pixel_to_meter_mapping(selected)

# Extract commonly used variables for backwards compatibility
pixels_to_meters_avg = mapping_info['pixels_to_meters_avg']
image_shape = mapping_info['image_shape'] 
sonar_coverage_meters = mapping_info['sonar_coverage_meters']

print(f"Using pixels_to_meters_avg = {pixels_to_meters_avg:.6f} m/px")
print(f"Mapping source: {mapping_info['source']}")

Using pixels_to_meters_avg = 0.016765 m/px
Mapping source: npz_metadata


In [8]:
iau.VisualizationEngine.plot_distance_analysis(net_analysis_results, "Real-World Distance Analysis")

In [12]:
# COMPARISON: SONAR vs DVL DISTANCE MEASUREMENTS
# =================================================
import utils.distance_measurement as sda

# IMPORTANT: Pass the by_bag folder, not just the exports root
# The function expects the folder containing the CSV files
from utils.config import EXPORTS_SUBDIRS
BY_BAG_FOLDER = EXPORTS_FOLDER / EXPORTS_SUBDIRS.get('by_bag', 'by_bag')

# Load all distance data for the target bag
raw_data, distance_measurements = sda.load_all_distance_data_for_bag(TARGET_BAG, BY_BAG_FOLDER)

# Display what we loaded
print(f"\n RAW DATA LOADED:")
for key, data in raw_data.items():
    if data is not None:
        print(f"    {key}: {len(data)} records")
    else:
        print(f"    {key}: None")

print(f"\n DISTANCE MEASUREMENTS LOADED:")
for key, info in distance_measurements.items():
    data_len = len(info['data'])
    print(f"    {key}: {data_len} records - {info['description']}")

 LOADING ALL DISTANCE DATA FOR BAG: 2024-08-20_14-31-29
 1. Loading Navigation Data...
    Loaded 559 navigation records
    NetPitch data available: 559 valid records
 2. Loading Guidance Data...
    Loaded 549 guidance records with ['error_net_distance', 'desired_net_distance', 'r_net_distance_d']
 3. Loading DVL Altimeter...
    DVL altimeter file not found
 4. Loading USBL...
    Loaded 33 USBL records
 5. Loading DVL Position...
    Loaded 283 DVL position records
 6. Loading Navigation Position...
    Loaded 278 navigation position records
 7. Loading INS Z Position...
    INS file not found

 LOADING SUMMARY:
    Target bag: 2024-08-20_14-31-29
    Raw data loaded: 2/2
    Distance measurements: 4

 RAW DATA LOADED:
    navigation: 559 records
    guidance: 549 records

 DISTANCE MEASUREMENTS LOADED:
    USBL_3D: 33 records - 3D acoustic position
    USBL_Depth: 33 records - USBL depth measurement
    DVL_Position: 283 records - 3D DVL position
    Nav_Position: 278 records - 2D

In [13]:
fig, comparison_stats = iau.ComparisonEngine.compare_sonar_vs_dvl(net_analysis_results, raw_data, sonar_coverage_m=sonar_coverage_meters, sonar_image_size=image_shape[0])
fig.show()


Comparison: Sonar=2.009m, DVL=1.968m, Ratio=1.021x


In [14]:
# Generate Three-SYSTEM Video 
# ===========================
from pathlib import Path
import importlib

# CONFIGURATION
FFT_CSV_PATH = Path("/Volumes/LaCie/SOLAQUA/relative_fft_pose")
TARGET_FFT_FILE = FFT_CSV_PATH / f"{TARGET_BAG}_relative_pose_fft.csv"

print("GENERATING THREE-SYSTEM VIDEO WITH UNIFORM STYLING")
print("=" * 60)
print(f"Target Bag: {TARGET_BAG}")

# Reload the module to pick up the new functions
import utils.video_generation as sg
importlib.reload(sg)

# Import the utility function
from utils.video_generation import generate_three_system_video

try:
    # Generate video using the streamlined utility function
    video_path = generate_three_system_video(
        target_bag=TARGET_BAG,
        exports_folder=EXPORTS_FOLDER,
        net_analysis_results=net_analysis_results,
        raw_data=raw_data,
        fft_csv_path=TARGET_FFT_FILE if TARGET_FFT_FILE.exists() else None,
        start_idx=1,
        end_idx=1500
    )
    
    print(f"\nVIDEO SUMMARY:")
    print(f"   File: {video_path.name}")
    print(f"   Location: {video_path.parent}")
    print(f"   Frames: 1-1500")
    print(f"   Systems: DVL (Yellow/Orange) + FFT (Cyan) + Sonar (Magenta)")
    print(f"   Styling: Uniform line thickness and visual appearance")
    
except Exception as e:
    print(f"ideo generation failed: {e}")
    print(f"Error details: {type(e).__name__}: {str(e)}")
    import traceback
    traceback.print_exc()

GENERATING THREE-SYSTEM VIDEO WITH UNIFORM STYLING
Target Bag: 2024-08-20_14-31-29
GENERATING TWO-SYSTEM VIDEO
🔄 PREPARING THREE-SYSTEM SYNCHRONIZED DATA
ℹ️  No FFT data path provided
   Running in two-system mode: DVL + Sonar only
✓ Using fallback synchronization method
✓ Fallback synchronization completed
   Systems active: DVL + Sonar
OPTIMIZED SONAR VIDEO WITH THREE-SYSTEM SYNCHRONIZATION
Target Bag: 2024-08-20_14-31-29
   Cone Size: 900x700
   Range: 0.0-5.0m | FOV: 120.0
Auto-detecting video files for bag: 2024-08-20_14-31-29
Found video frames: 2024-08-20_14-31-29_video__image_compressed_image_data_frames
Using frames directory: /Volumes/LaCie/SOLAQUA/exports/frames/2024-08-20_14-31-29_video__image_compressed_image_data_frames
Camera: enabled
Net-line: enabled (dist tol=0.5s, pitch tol=2.0s)
Sonar Analysis: enabled
FFT Data: disabled
   Loading sonar data: sensor_sonoptix_echo_image__2024-08-20_14-31-29_video.csv
Loaded 1060 sonar frames in 41.93s
 LOADING ALL DISTANCE DATA FOR 